## The stated goal is to predict daily demand with time, temperature and holiday information 
using various scikit-learn methods. 

For this kind of problem we need a target variable and a whole lot of normalized predictor variables.


In [1]:
import pandas as pd

# read data fetched in the previous step
demand_df       = pd.read_pickle("dataframes/demand_df.pickle.gz", compression="infer")
daily_demand_df = pd.read_pickle("dataframes/daily_demand_df.pickle.gz", compression="infer")
weather_df      = pd.read_pickle("dataframes/weather_df.pickle.gz", compression="infer")
holiday_df      = pd.read_pickle("dataframes/holiday_df.pickle.gz", compression="infer")


## Create one-hot weekday list
using the target list's date field

In [2]:
dayofweek_onehot = daily_demand_df[['opday']]
dayofweek_onehot['is'] = dayofweek_onehot.opday.dt.day_name()
dayofweek_onehot = pd.get_dummies(dayofweek_onehot.set_index('opday'))
dayofweek_onehot

,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday
opday,,,,,,,
2015-10-01,0,0,0,0,1,0,0
2015-10-02,1,0,0,0,0,0,0
2015-10-03,0,0,1,0,0,0,0
2015-10-04,0,0,0,1,0,0,0
2015-10-05,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
2019-12-28,0,0,1,0,0,0,0
2019-12-29,0,0,0,1,0,0,0
2019-12-30,0,1,0,0,0,0,0


In [3]:
holiday_df.dtypes

holiday    object
dtype: object

## Create one-hot holiday list

In [4]:
holiday_onehot = pd.get_dummies(holiday_df)
holiday_onehot['is_holiday'] = holiday_onehot.sum(axis=1)
holiday_onehot

,holiday_Christmas Day,holiday_Christmas Day (Observed),holiday_Christmas Eve,holiday_Day After Thanksgiving,holiday_Independence Day,holiday_Independence Day (Observed),holiday_Labor Day,holiday_Memorial Day,holiday_New Year's Day,holiday_New Year's Day (Observed),holiday_Thanksgiving,is_holiday
date,,,,,,,,,,,,
2015-01-01,0,0,0,0,0,0,0,0,1,0,0,1
2015-05-25,0,0,0,0,0,0,0,1,0,0,0,1
2015-07-03,0,0,0,0,0,1,0,0,0,0,0,1
2015-07-04,0,0,0,0,1,0,0,0,0,0,0,1
2015-09-07,0,0,0,0,0,0,1,0,0,0,0,1
2015-11-26,0,0,0,0,0,0,0,0,0,0,1,1
2015-11-27,0,0,0,1,0,0,0,0,0,0,0,1
2015-12-24,0,0,1,0,0,0,0,0,0,0,0,1
2015-12-25,1,0,0,0,0,0,0,0,0,0,0,1


## Get the weather DF into a single row per day with all predictor attributes

In [5]:
for station in weather_df.STATION.unique():
    print (station)

USW00013967
USW00024011
USW00014939
USW00023047
USW00013996
USW00013968


## The original analysis summed the HDD and CDD columns over all weather stations

In [6]:
np_sum_cdd = weather_df.pivot(index=['DATE'], columns='STATION', values=['CDD']).sum(axis=1)
sum_cdd_df = pd.DataFrame(np_sum_cdd, columns = ['SUM_CDD'])
np_sum_hdd = weather_df.pivot(index=['DATE'], columns='STATION', values=['HDD']).sum(axis=1)
sum_hdd_df = pd.DataFrame(np_sum_hdd, columns = ['SUM_HDD'])
sum_cdd_df

,SUM_CDD
DATE,
2015-01-01,249.66
2015-01-02,216.54
2015-01-03,217.08
2015-01-04,291.42
2015-01-05,295.74
...,...
2021-10-14,50.88
2021-10-15,57.60
2021-10-16,76.14


In [7]:
# pivot out all weather values by day; append station name to column name:
weather_pivot = weather_df.pivot(index=['DATE'], columns='STATION', values=['TMIN', 'TMAX', 'TAVG','CDD','HDD'])
weather_pivot.columns = ['_'.join(col).strip() for col in weather_pivot.columns.values]
# merge in CDD and HDD sums
weather_pivot = weather_pivot.merge(sum_cdd_df, on='DATE').merge(sum_hdd_df, on='DATE')
weather_pivot


,TMIN_USW00013967,TMIN_USW00013968,TMIN_USW00013996,TMIN_USW00014939,TMIN_USW00023047,TMIN_USW00024011,TMAX_USW00013967,TMAX_USW00013968,TMAX_USW00013996,TMAX_USW00014939,...,CDD_USW00023047,CDD_USW00024011,HDD_USW00013967,HDD_USW00013968,HDD_USW00013996,HDD_USW00014939,HDD_USW00023047,HDD_USW00024011,SUM_CDD,SUM_HDD
DATE,,,,,,,,,,,,,,,,,,,,,
2015-01-01,-4.3,-3.2,-8.2,-13.2,-8.2,-4.3,1.1,-0.5,2.2,1.7,...,45.24,38.76,0.0,0.00,0.0,0.0,0.0,0.0,249.66,0.00
2015-01-02,-0.5,-0.5,-1.6,-11.6,-7.7,-7.1,2.8,3.3,3.3,0.6,...,39.48,37.32,0.0,0.00,0.0,0.0,0.0,0.0,216.54,0.00
2015-01-03,1.7,1.7,-8.8,-12.7,-7.7,-19.9,5.6,6.1,5.6,3.3,...,36.42,52.44,0.0,0.00,0.0,0.0,0.0,0.0,217.08,0.00
2015-01-04,-7.7,-8.8,-13.2,-17.7,-12.7,-24.9,1.1,3.3,-7.7,-11.6,...,44.16,69.36,0.0,0.00,0.0,0.0,0.0,0.0,291.42,0.00
2015-01-05,-8.8,-9.3,-14.9,-14.3,-7.1,-28.2,6.7,5.6,-2.7,-3.2,...,35.52,73.68,0.0,0.00,0.0,0.0,0.0,0.0,295.74,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-14,12.2,16.7,6.1,1.1,3.9,4.4,23.9,22.8,22.8,18.3,...,9.24,21.12,0.0,1.56,0.0,0.0,0.0,0.0,50.88,1.56
2021-10-15,8.3,9.4,5.6,3.9,2.2,0.0,19.4,21.1,18.9,17.8,...,15.36,19.68,0.0,0.00,0.0,0.0,0.0,0.0,57.60,0.00
2021-10-16,5.6,6.1,4.4,3.9,1.7,-0.6,21.1,22.2,20.0,20.0,...,14.46,17.52,0.0,0.00,0.0,0.0,0.0,0.0,76.14,0.00


## Create a target dataframe with the target variable and all the potential predictors

In [8]:
# merge daily_demand_f and dayofweek_onehot

target_df = daily_demand_df.merge(dayofweek_onehot, on='opday')
target_df

,opday,sum_spp_load,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday
0,2015-10-01,502184.942993,0,0,0,0,1,0,0
1,2015-10-02,612695.032412,1,0,0,0,0,0,0
2,2015-10-03,566229.834662,0,0,1,0,0,0,0
3,2015-10-04,552166.723416,0,0,0,1,0,0,0
4,2015-10-05,596751.461994,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1549,2019-12-28,648146.557000,0,0,1,0,0,0,0
1550,2019-12-29,661993.471000,0,0,0,1,0,0,0
1551,2019-12-30,752393.427000,0,1,0,0,0,0,0
1552,2019-12-31,746139.564000,0,0,0,0,0,1,0


In [9]:
# append holiday_onehot

target_df = target_df.merge(holiday_onehot, how = 'left', left_on='opday', right_on = 'date').fillna(0, downcast="infer")
target_df

,opday,sum_spp_load,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday,holiday_Christmas Day,...,holiday_Christmas Eve,holiday_Day After Thanksgiving,holiday_Independence Day,holiday_Independence Day (Observed),holiday_Labor Day,holiday_Memorial Day,holiday_New Year's Day,holiday_New Year's Day (Observed),holiday_Thanksgiving,is_holiday
0,2015-10-01,502184.942993,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-10-02,612695.032412,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-10-03,566229.834662,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-10-04,552166.723416,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-10-05,596751.461994,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,2019-12-28,648146.557000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1550,2019-12-29,661993.471000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1551,2019-12-30,752393.427000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1552,2019-12-31,746139.564000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# append weather_pivot

target_df = target_df.merge(weather_pivot, left_on='opday', right_on = 'DATE')
target_df

,opday,sum_spp_load,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday,holiday_Christmas Day,...,CDD_USW00023047,CDD_USW00024011,HDD_USW00013967,HDD_USW00013968,HDD_USW00013996,HDD_USW00014939,HDD_USW00023047,HDD_USW00024011,SUM_CDD,SUM_HDD
0,2015-10-01,502184.942993,0,0,0,0,1,0,0,0,...,0.00,3.48,0.0,0.0,0.0,0.0,8.76,0.0,20.10,8.76
1,2015-10-02,612695.032412,1,0,0,0,0,0,0,0,...,1.14,9.78,0.0,0.0,0.0,0.0,0.00,0.0,38.88,0.00
2,2015-10-03,566229.834662,0,0,1,0,0,0,0,0,...,11.94,14.46,0.0,0.0,0.0,0.0,0.00,0.0,61.38,0.00
3,2015-10-04,552166.723416,0,0,0,1,0,0,0,0,...,13.20,14.46,0.0,0.0,0.0,0.0,0.00,0.0,55.62,0.00
4,2015-10-05,596751.461994,0,1,0,0,0,0,0,0,...,9.06,13.02,0.0,0.0,0.0,0.0,0.00,0.0,52.56,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,2019-12-28,648146.557000,0,0,1,0,0,0,0,0,...,21.48,48.84,0.0,0.0,0.0,0.0,0.00,0.0,132.12,0.00
1550,2019-12-29,661993.471000,0,0,0,1,0,0,0,0,...,32.10,45.42,0.0,0.0,0.0,0.0,0.00,0.0,169.74,0.00
1551,2019-12-30,752393.427000,0,1,0,0,0,0,0,0,...,34.44,43.26,0.0,0.0,0.0,0.0,0.00,0.0,197.10,0.00
1552,2019-12-31,746139.564000,0,0,0,0,0,1,0,0,...,31.38,49.74,0.0,0.0,0.0,0.0,0.00,0.0,199.08,0.00


In [11]:
# one station doesn't have data for one day; just exclude that one row. 
target_df.dropna(inplace=True)

# save the complete target dataframe
target_df.to_pickle("dataframes/target_df.pickle.gz", compression="infer")
target_df.count()

opday                                  1553
sum_spp_load                           1553
is_Friday                              1553
is_Monday                              1553
is_Saturday                            1553
is_Sunday                              1553
is_Thursday                            1553
is_Tuesday                             1553
is_Wednesday                           1553
holiday_Christmas Day                  1553
holiday_Christmas Day (Observed)       1553
holiday_Christmas Eve                  1553
holiday_Day After Thanksgiving         1553
holiday_Independence Day               1553
holiday_Independence Day (Observed)    1553
holiday_Labor Day                      1553
holiday_Memorial Day                   1553
holiday_New Year's Day                 1553
holiday_New Year's Day (Observed)      1553
holiday_Thanksgiving                   1553
is_holiday                             1553
TMIN_USW00013967                       1553
TMIN_USW00013968                

## Normalize everything
per https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame

In [12]:
normalized_target_df=(target_df-target_df.min())/(target_df.max()-target_df.min())

In [13]:
normalized_target_df

,opday,sum_spp_load,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday,holiday_Christmas Day,...,CDD_USW00023047,CDD_USW00024011,HDD_USW00013967,HDD_USW00013968,HDD_USW00013996,HDD_USW00014939,HDD_USW00023047,HDD_USW00024011,SUM_CDD,SUM_HDD
0,0.000000,0.383577,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.040503,0.0,0.0,0.0,0.0,0.345972,0.0,0.052898,0.067034
1,0.000644,0.516002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.019833,0.113827,0.0,0.0,0.0,0.0,0.0,0.0,0.102321,0.0
2,0.001288,0.460323,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.207724,0.168296,0.0,0.0,0.0,0.0,0.0,0.0,0.161535,0.0
3,0.001932,0.443471,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.229645,0.168296,0.0,0.0,0.0,0.0,0.0,0.0,0.146376,0.0
4,0.002576,0.496897,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15762,0.151536,0.0,0.0,0.0,0.0,0.0,0.0,0.138323,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,0.997424,0.558484,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.373695,0.568436,0.0,0.0,0.0,0.0,0.0,0.0,0.347703,0.0
1550,0.998068,0.575077,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.558455,0.528631,0.0,0.0,0.0,0.0,0.0,0.0,0.446708,0.0
1551,0.998712,0.683404,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.599165,0.503492,0.0,0.0,0.0,0.0,0.0,0.0,0.518712,0.0
1552,0.999356,0.67591,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.545929,0.578911,0.0,0.0,0.0,0.0,0.0,0.0,0.523922,0.0


In [14]:
normalized_target_df.to_pickle("dataframes/normalized_target_df.pickle.gz", compression="infer")